In [1]:
import pandas as pd
import numpy  as np

   
import pickle
   

from scipy.sparse import issparse

 
from pathlib import Path


pd.set_option('display.max_columns', 5)

In [2]:
#### user specified
data_folder = r"D:/analyze_Pearson_residuals/"

data_subfolder = "retinal"

data_path = Path ( data_folder + data_subfolder )

In [3]:
# output data
results_pkl = "Lg_complementary_samples.pkl"

# input data
counts_pkl = "counts_sparse.pkl" # sparse pandas data frame: cells in columns; genes in rows
gene_array_pkl  = "gene_array_complementary_samples.pkl"
df_cells_pkl  = "df_cells_complementary_samples.pkl" 


# path: output data
results_dsn = data_path / results_pkl

# paths: input data
counts_dsn = data_path / counts_pkl  
df_cells_dsn = data_path / df_cells_pkl
gene_array_dsn = data_path / gene_array_pkl

In [4]:
# https://github.com/scverse/scanpy/tree/master/scanpy/experimental/pp
# from   _normalization.py  rows 26-64

def _pearson_residuals(X, theta, clip, copy=False):    # removed check_values from parm list

    X = X.copy() if copy else X

    # check theta
    if theta <= 0:
        # TODO: would "underdispersion" with negative theta make sense?
        # then only theta=0 were undefined..
        raise ValueError('Pearson residuals require theta > 0')
    # prepare clipping
    if clip is None:
        n = X.shape[0]
        clip = np.sqrt(n)
    if clip < 0:
        raise ValueError("Pearson residuals require `clip>=0` or `clip=None`.")

# vfk 2022 08 16		
    # if check_values and not check_nonnegative_integers(X):
        # warn(
            # "`normalize_pearson_residuals()` expects raw count data, but non-integers were found.",
            # UserWarning,
        # )

    if issparse(X):
        sums_genes = np.sum(X, axis=0)
        sums_cells = np.sum(X, axis=1)
        sum_total = np.sum(sums_genes).squeeze()
    else:
        sums_genes = np.sum(X, axis=0, keepdims=True)
        sums_cells = np.sum(X, axis=1, keepdims=True)
        sum_total = np.sum(sums_genes)

    mu = np.array(sums_cells @ sums_genes / sum_total)
    diff = np.array(X - mu)
    residuals = diff / np.sqrt(mu + mu**2 / theta)

    # clip
    residuals = np.clip(residuals, a_min=-clip, a_max=clip)

    return residuals	
	
	
pctl_list = [.01,.05, .10, .25, .5, .75, .90, .95, .96, .97, .98, .99 ]

In [5]:
df_selected_cells = pd.read_pickle( df_cells_dsn )
print ( ' df_selected_cells:  ' )
print ( '\n', df_selected_cells )
print ( '\n\n df_selected_cells.sum:  ' )
print ( df_selected_cells.sum() )

f = open( gene_array_dsn, 'rb' )    
gene_select_array  = pickle.load(f)     
f.close()       
print ( '\n\n gene_select_array:  ' )
print ( gene_select_array )

 df_selected_cells:  

                      0      1
r4_TGCCACATGGGC  False   True
r4_ATCGGCTACCGA   True  False
r4_TAGATATCTTAT   True  False
r4_CCTGGATTGTAC   True  False
r4_CGGTAATAGGAA  False   True
...                ...    ...
p1_TCAAAAGCCGGG  False   True
p1_ATTAAGTTCCAA  False   True
p1_CTGTCTGAGACC   True  False
p1_TAACGCGCTCCT   True  False
p1_ATTCTTGTTCTT   True  False

[24769 rows x 2 columns]


 df_selected_cells.sum:  
0    12448
1    12321
dtype: int64


 gene_select_array:  
['CARTPT' 'RGS5' 'RHO' ... 'SHB' 'IL34' 'MRAP2']


In [6]:
df_counts_sparse = pd.read_pickle ( counts_dsn )
print ( ' df_counts_sparse:' )
print ( df_counts_sparse )
print ( '\n type( df_counts_sparse ) ', type ( df_counts_sparse ))
print ( '\n df_counts_sparse.sparse.density: ', df_counts_sparse.sparse.density)

 df_counts_sparse:
               r4_TGCCACATGGGC  r4_ATCGGCTACCGA  ...  p1_TAACGCGCTCCT  \
KITL                         0                2  ...                0   
TMTC3                       10                0  ...                0   
CEP290                      24                8  ...                0   
4930430F08RIK                1                1  ...                0   
1700017N19RIK                0                1  ...                0   
...                        ...              ...  ...              ...   
GM23791                      0                0  ...                0   
GM26101                      0                0  ...                0   
4930513O06RIK                0                0  ...                0   
VSIG1                        0                0  ...                0   
GM25207                      0                0  ...                0   

               p1_ATTCTTGTTCTT  
KITL                         0  
TMTC3                        0  
CEP29

In [7]:
df_counts_sparse_all_samples_SD_positive = df_counts_sparse[ df_selected_cells.index ].loc [ gene_select_array ]
print ( ' df_counts_sparse_all_samples_SD_positive:' )
print ( df_counts_sparse_all_samples_SD_positive )
print ( '\n type( df_counts_sparse_all_samples_SD_positive ) ', type ( df_counts_sparse_all_samples_SD_positive ))
print ( '\n df_counts_sparse_all_samples_SD_positive.sparse.density: ', df_counts_sparse_all_samples_SD_positive.sparse.density)

del df_counts_sparse

 df_counts_sparse_all_samples_SD_positive:
         r4_TGCCACATGGGC  r4_ATCGGCTACCGA  ...  p1_TAACGCGCTCCT  \
CARTPT                 0               45  ...                3   
RGS5                   0                0  ...                0   
RHO                  496                5  ...               32   
APOE                   0                0  ...                0   
GLUL                   7                4  ...                0   
...                  ...              ...  ...              ...   
GM26906                0                1  ...                0   
LHFPL5                 0                2  ...                0   
SHB                    0                0  ...                0   
IL34                   0                4  ...                0   
MRAP2                  0                0  ...                0   

         p1_ATTCTTGTTCTT  
CARTPT                 0  
RGS5                   0  
RHO                   35  
APOE                   0  
GLUL             

In [8]:
df_clipped_half_cell_samples_list = []

for sample in  range(2):
  print ( '\n sample: ', sample )
  df_cell_sample = df_selected_cells[[sample]]
  df_cell_select = df_cell_sample [ df_cell_sample[sample] ]
  barcode_list = df_cell_select.index.values.tolist()
  
  df_counts_sparse_sample = df_counts_sparse_all_samples_SD_positive[ barcode_list ]     
  scipy_csr_mat = df_counts_sparse_sample.transpose().sparse.to_coo().tocsr(). astype ( 'float64'  )

  residuals = _pearson_residuals ( scipy_csr_mat, 100, None )
  residual_variance_clipped = np.var( residuals, axis=0)

  df_clipped_half_cell_sample = pd.DataFrame ( index = df_counts_sparse_all_samples_SD_positive.index, data = { sample: residual_variance_clipped  } ) 
  df_clipped_half_cell_sample.sort_values ( [sample], ascending=False, inplace=True )  

  df_clipped_half_cell_samples_list.append ( df_clipped_half_cell_sample )
  
  
df_clipped_half_cell_samples = pd.concat ( df_clipped_half_cell_samples_list, axis=1 )
print ( '\n df_clipped_half_cell_samples' )
print ( df_clipped_half_cell_samples  )
print ( '\n\n df_clipped_half_cell_samples.describe' )
print ( df_clipped_half_cell_samples.describe ( percentiles=pctl_list ) )


 sample:  0

 sample:  1

 df_clipped_half_cell_samples
                   0          1
CARTPT     34.868703  28.818525
RGS5       21.454272  16.127682
APOE       17.131418  15.826166
GLUL       15.464545  14.996507
MT-RNR2    14.496866  13.857703
...              ...        ...
IL34        0.501279   0.455905
SLC41A3     0.499102   0.690357
FABP3-PS1   0.489785   0.475354
SHB         0.486703   0.570099
GM26906     0.469195   0.619259

[12082 rows x 2 columns]


 df_clipped_half_cell_samples.describe
                  0             1
count  12082.000000  12082.000000
mean       1.230584      1.229248
std        0.690012      0.648922
min        0.469195      0.455905
1%         0.659494      0.659500
5%         0.794920      0.800744
10%        0.880452      0.886095
25%        1.021362      1.023161
50%        1.152347      1.151923
75%        1.311100      1.310324
90%        1.518166      1.509388
95%        1.698940      1.688767
96%        1.770580      1.758730
97%        1.911

In [9]:
df_clipped_half_cell_samples.to_pickle ( results_dsn )